In [ ]:
{-# LANGUAGE MonomorphismRestriction #-}

import System.Random
import Control.Monad.State

In [ ]:
type RandomState = State StdGen

getRandom :: Random a => RandomState a
getRandom = do
    g <- get
    let (v, g') = random g
    put g'
    return v

getTwoRandoms :: Random a => RandomState (a, a)
getTwoRandoms = liftM2 (,) getRandom getRandom

In [ ]:
: StdGen

In [ ]:
runTwoRandoms :: IO (Int, Int)
runTwoRandoms = do
    s <- getStdGen
    let (r, s') = runState getTwoRandoms s
    setStdGen s'
    return r

runTwoRandoms >>= print